In [1]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

%matplotlib inline

In [2]:
#since dataset is huge only importing necessary columns 
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9', usecols = ['Unique Key','Agency', 'Complaint Type', 'Community Board', 'Borough','Created Date', 'Closed Date', 'Resolution Description'])

In [3]:
data.head()

,Unique Key,Created Date,Closed Date,Agency,Complaint Type,Resolution Description,Community Board,Borough
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Street Condition,NaN,13 BROOKLYN,BROOKLYN
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,Opinion for the Mayor,Your comments have been submitted to the Mayor...,0 Unspecified,Unspecified
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Root/Sewer/Sidewalk Condition,The Department of Parks and Recreation usually...,18 BROOKLYN,BROOKLYN
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,Illegal Parking,Your complaint has been received by the Police...,12 BROOKLYN,BROOKLYN
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,Benefit Card Replacement,The Human Resources Administration received yo...,0 Unspecified,Unspecified


In [5]:
data.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Complaint Type',
       'Resolution Description', 'Community Board', 'Borough'],
      dtype='object')

In [10]:
# Check if all Community Districts are represented in the Data?
len(data["Community Board"].unique())

77

In [11]:
# Check if all Boroughs are represented in the Data?
data["Borough"].unique()

array(['BROOKLYN', 'Unspecified', 'STATEN ISLAND', 'MANHATTAN', 'BRONX',
       'QUEENS'], dtype=object)

In [7]:
# Check if all Community Districts are represented in the Data?
data["Community Board"].unique()

array(['13 BROOKLYN', '0 Unspecified', '18 BROOKLYN', '12 BROOKLYN',
       '02 STATEN ISLAND', '08 MANHATTAN', '06 BROOKLYN', '10 BRONX',
       '06 MANHATTAN', '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN',
       '14 BROOKLYN', '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS',
       '07 QUEENS', '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN',
       '09 QUEENS', '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN',
       '03 BRONX', '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN',
       '07 MANHATTAN', '10 MANHATTAN', '04 BRONX', '01 MANHATTAN',
       '07 BRONX', '09 BRONX', '03 MANHATTAN', '13 QUEENS', '02 BRONX',
       '12 QUEENS', '16 BROOKLYN', '08 QUEENS', '04 MANHATTAN',
       '14 QUEENS', '10 BROOKLYN', '11 QUEENS', '06 BRONX', '08 BRONX',
       '05 BRONX', '15 BROOKLYN', '02 BROOKLYN', '12 BRONX', '03 QUEENS',
       '06 QUEENS', '01 STATEN ISLAND', '01 BRONX', '01 QUEENS',
       '17 BROOKLYN', '11 BRONX', 'Unspecified QUEENS', '09 BROOKLYN',
       'Unspecified MANHATTAN'

In [9]:
# Check if all Boroughs are represented in the Data?
data["Borough"].unique()

array(['BROOKLYN', 'Unspecified', 'STATEN ISLAND', 'MANHATTAN', 'BRONX',
       'QUEENS'], dtype=object)

In [8]:
#All of the boroughs are represented
data.Borough.value_counts()

BROOKLYN         2831932
QUEENS           2189760
MANHATTAN        1900005
BRONX            1665625
Unspecified      1109446
STATEN ISLAND     490998
Name: Borough, dtype: int64

In [ ]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.



We have 77 because of the 311 system collection methods.  There are supposed to be 59 community Districts.  In addition it does not appear that all of the districts are represented. I am not sure what the number are before the districts, the conventional system is indicated here: http://www.fcny.org/cmgp/streets/pages/nyc_cdmap4.htm

In [17]:
# Check for duplicates? Are these plausible?


In [16]:
uniquevalues = data.drop_duplicates()

In [18]:
len(data)

10187766

In [19]:
len(uniquevalues)

10187766

There are no duplicates

In [20]:
# Generate marker for unplausible Community Districts.  What do these districts look like?
markerQueens = data['Community Board'].str.contains('8[0-9] QUEENS')
#80,81,82,83,84
markerBrooklyn = data['Community Board'].str.contains('5[0-9] BROOKLYN')
#55,56
markerBronx = data['Community Board'].str.contains('2[0-9] BRONX')
#26,27,28
markerManhattan = data['Community Board'] == '64 MANHATTAN'
#64
markerStaten = data['Community Board'] == '95 STATEN ISLAND'
#95
markerUnsp = data['Community Board'].str.contains('Unspecified')

makgerGeneral = (~markerQueens) & (~markerBronx) & (~markerBrooklyn) & (~markerManhattan) & (~markerStaten) & (~markerUnsp)

In [21]:
# Drop the marked districts
data2 = data.loc[makgerGeneral,]
data2.shape

(8110945, 8)

In [22]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level
data2[['Unique Key','Community Board']].groupby('Community Board').count()

,Unique Key
Community Board,
01 BRONX,74631
01 BROOKLYN,185057
01 MANHATTAN,77974
01 QUEENS,171484
01 STATEN ISLAND,182713
02 BRONX,60257
02 BROOKLYN,121022
02 MANHATTAN,133860
02 QUEENS,114333


In [ ]:
# Safe reduced data frame (Community District level)

I am not sure what this means.

4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data.

0

In [32]:
# Check variables in file
df_pop.columns

Index(['FIPS', 'cd_id', 'Total Population',
       'Population Density (per sq. mile)', '% Total Population: Male',
       '% Total Population: 18 to 24 Years',
       '% Total Population: 25 to 34 Years',
       '% Total Population: 35 to 44 Years',
       '% Population 5 Years And Over: Speak Only English',
       '% Population 5 Years And Over: Spanish or Spanish Creole',
       ...
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Production occupations',
       'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Transportation and material moving occupations',
       '% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations',
       '% Employed Civilian Population 16 Years And Over: Service occupations',
       '% Employed Civilian Population 16 Years And Over: Sales and office occupations',
       '% Employed Civ

In [33]:
# Check variables in file
df_pop.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [34]:
# How many community districts are in file? 
df_pop["cd_id"].unique().shape

(59,)

In [35]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311



In [36]:
# Infrastructure by Community District

df_infr = pd.read_csv("http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [37]:
# Check variables in file

df_infr.columns

Index(['FIPS', 'Geographic Identifier', 'Qualifying Name', 'Households',
       'Households: With An Internet Subscription',
       'Households: Dial-Up Alone', 'Households: Dsl',
       'Households: With Mobile Broadband',
       'Households: Without Mobile Broadband', 'Households: Cable Modem',
       'Households: With Mobile Broadband.1',
       'Households: Without Mobile Broadband.1', 'Households: Fiber-Optic',
       'Households: With Mobile Broadband.2',
       'Households: Without Mobile Broadband.2',
       'Households: Satellite Internet Service',
       'Households: With Mobile Broadband.3',
       'Households: Without Mobile Broadband.3',
       'Households: Two or More Fixed Broadband Types, or Other',
       'Households: With Mobile Broadband.4',
       'Households: Without Mobile Broadband.4',
       'Households: Mobile Broadband Alone or With Dialup',
       'Households: Internet Access Without A Subscription',
       'Households: No Internet Access',
       '% Househol

In [38]:
df_infr.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [39]:
# How many community districts are in file? 
df_infr["Qualifying Name"].unique().shape

(55,)

In [ ]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data

In [ ]:

# Aggregate internet type by high and low connections


In [41]:
# create new dataframe
df_inf_percent = pd.DataFrame()
df_inf_percent['Qualifying Name'] = df_infr['Qualifying Name']
df_inf_percent['% Households: With An Internet Subscription'] = df_infr['% Households: With An Internet Subscription']



In [42]:
df_inf_percent['% with mobile'] = (df_infr['Households: With Mobile Broadband'] + 
                               df_infr['Households: With Mobile Broadband.1'] +
                               df_infr['Households: With Mobile Broadband.2'] + 
                               df_infr['Households: With Mobile Broadband.3'] + 
                               df_infr['Households: With Mobile Broadband.4'])/df_infr.Households*100

df_inf_percent['% without mobile'] = (df_infr['Households: Without Mobile Broadband'] + 
                                  df_infr['Households: Without Mobile Broadband.1'] +
                                  df_infr['Households: Without Mobile Broadband.2'] + 
                                  df_infr['Households: Without Mobile Broadband.3'] + 
                                  df_infr['Households: Without Mobile Broadband.4'])/df_infr.Households*100

df_inf_percent['% fast connection'] = (df_infr['Households: Dsl'] + df_infr['Households: Cable Modem'] +
                                   df_infr['Households: Fiber-Optic'] + 
                                   df_infr['Households: Satellite Internet Service'] + 
                                   df_infr['Households: Two or More Fixed Broadband Types, or Other'])/df_infr.Households*100

df_inf_percent['% slow connection'] = (df_infr['Households: Dial-Up Alone'] + 
                                   df_infr['Households: Mobile Broadband Alone or With Dialup'] +
                                   df_infr['Households: Internet Access Without A Subscription'])/df_infr.Households*100

df_inf_percent['Qualifying Name'] = df_infr['Qualifying Name']


df_inf_percent.head()

In [44]:

# Harmonize identifier of dataframe 1
data.head()

,Unique Key,Created Date,Closed Date,Agency,Complaint Type,Resolution Description,Community Board,Borough
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Street Condition,NaN,13 BROOKLYN,BROOKLYN
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,Opinion for the Mayor,Your comments have been submitted to the Mayor...,0 Unspecified,Unspecified
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Root/Sewer/Sidewalk Condition,The Department of Parks and Recreation usually...,18 BROOKLYN,BROOKLYN
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,Illegal Parking,Your complaint has been received by the Police...,12 BROOKLYN,BROOKLYN
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,Benefit Card Replacement,The Human Resources Administration received yo...,0 Unspecified,Unspecified


Merge the three dataframes, using a combination of the community board column and the borough column

In [ ]:
# Harmonize identifier of dataframe 2

In [45]:
df_pop.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [ ]:
# Harmonize identifier of dataframe 3

In [46]:
df_infr.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [ ]:
# Link the 3 dataframes